# Лаб 2. Матричные разложения

## Теория

Суть методов, основанных на матричном разложении состоит в том, что каждому пользователю $u$ ставится в соответствие вектор интересов $p_u$ длины $k$, а каждому айтему $i$ ставится в соответствие вектор интересов $q_i$, которым он удовлетворяет. Таким образом, что скалярное произведение этих векторов даёт предсказание $\hat r_{ui}$:

$$
\hat r_{ui} = p_u \dot q_i
$$

Такие вектора $p_u$ и $q_i$ составляют матрицы $P$ и $Q$ соответственно, которые в произведении даёт матрицу предсказаний $\hat R$.

![Разложение матриц](./images/PQ.drawio.png)

## Код

Ставим библиотеки

In [1]:
#%pip install --quiet -U scikit-surprise

Импорты

In [2]:
from tqdm.notebook import tqdm
import pandas as pd

# Реализации методов матричного разложения будем использовать из библиотеки surprise
from surprise import Dataset, SVD, NMF, Reader
# Кроссвалидацию проходили на машинном обучении
from surprise.model_selection import cross_validate

Загрузка датасета (из первой лабораторной)

In [3]:
data = Dataset.load_from_file('./ml-latest-small/ratings.csv', Reader(sep=',', line_format="user item rating timestamp", skip_lines=1))
trainset = data.build_full_trainset()

user_id = '42'
item_id = '50'

### SVD

In [4]:
# Обучаем модель
algo_svd = SVD(biased=False, n_factors=100)
algo_svd = algo_svd.fit(trainset)

In [5]:
# Даём предсказание для юзера и произвольного айтема
algo_svd.predict(user_id, item_id).est

4.5482913957517725

Матричное представление пользователей

In [6]:
algo_svd.pu.shape

(610, 100)

In [7]:
#algo_svd.pu

Матричное представление товаров

In [8]:
algo_svd.qi.shape

(9724, 100)

In [9]:
#algo_svd.qi

Пример связи матричных представлений и итогово предсказания

In [10]:
svd_item_id = algo_svd.trainset.to_inner_iid(item_id)
svd_user_id = algo_svd.trainset.to_inner_uid(user_id)

# Умножаем вектор интересов пользователя на вектор интересов айтема
(algo_svd.pu[svd_user_id]*algo_svd.qi[svd_item_id]).sum()

4.548291395751772

In [11]:
algo_svd.pu[svd_user_id].shape,algo_svd.qi[svd_item_id].shape,svd_item_id,svd_user_id

((100,), (100,), 4, 41)

### Non-negative Matrix Factorization

По интерфейсу и внутреннему устройству этот метод похож на SVD. Повторите предыдущие шаги для него.

In [12]:
algo_nmf = NMF(n_factors=15)
algo_nmf = algo_nmf.fit(trainset)

In [13]:
algo_nmf.predict(user_id, item_id).est

4.532347370256509

In [14]:
nmf_item_id = algo_nmf.trainset.to_inner_iid(item_id)
nmf_user_id = algo_nmf.trainset.to_inner_uid(user_id)

# Умножаем вектор интересов пользователя на вектор интересов айтема
(algo_nmf.pu[nmf_user_id]*algo_nmf.qi[nmf_item_id]).sum()

4.532347370256508

### Валидация и обучение
Используя кроссвалидацию, сравните представленные алгоритмы по качеству.

Подберите гиперпараметры моделей для улучшения результатов.

Расшифровка метрик ошибок и альтернативы можно найти в документации (как и параметры моделей): https://surprise.readthedocs.io/en/stable/accuracy.html#module-surprise.accuracy

In [15]:
cross_validate(algo_svd, data, verbose=True)

Evaluating RMSE, MAE of algorithm SVD on 5 split(s).

                  Fold 1  Fold 2  Fold 3  Fold 4  Fold 5  Mean    Std     
RMSE (testset)    0.9783  0.9796  0.9807  0.9785  0.9802  0.9795  0.0009  
MAE (testset)     0.7533  0.7548  0.7578  0.7523  0.7548  0.7546  0.0018  
Fit time          1.53    1.66    1.27    1.21    1.47    1.43    0.17    
Test time         0.32    0.12    0.14    0.25    0.18    0.20    0.08    


{'test_rmse': array([0.97827173, 0.97960082, 0.9806997 , 0.97851434, 0.98020074]),
 'test_mae': array([0.75333062, 0.75477521, 0.75778205, 0.7523072 , 0.75483536]),
 'fit_time': (1.529911994934082,
  1.6575686931610107,
  1.2676122188568115,
  1.2067744731903076,
  1.4660837650299072),
 'test_time': (0.3231353759765625,
  0.11668825149536133,
  0.13862872123718262,
  0.24933409690856934,
  0.17553210258483887)}

In [16]:
cross_validate(algo_nmf, data, verbose=True)

Evaluating RMSE, MAE of algorithm NMF on 5 split(s).

                  Fold 1  Fold 2  Fold 3  Fold 4  Fold 5  Mean    Std     
RMSE (testset)    0.9142  0.9198  0.9294  0.9209  0.9266  0.9222  0.0053  
MAE (testset)     0.6994  0.7061  0.7115  0.7069  0.7074  0.7063  0.0039  
Fit time          2.37    2.21    2.58    2.38    2.45    2.40    0.12    
Test time         0.11    0.20    0.12    0.11    0.20    0.15    0.04    


{'test_rmse': array([0.91417419, 0.91976225, 0.92938718, 0.92090248, 0.92656795]),
 'test_mae': array([0.6993649 , 0.70613254, 0.71154532, 0.70691851, 0.70739353]),
 'fit_time': (2.3686680793762207,
  2.2130846977233887,
  2.5761513710021973,
  2.3796396255493164,
  2.4474575519561768),
 'test_time': (0.10970711708068848,
  0.2014613151550293,
  0.11667919158935547,
  0.10970735549926758,
  0.19946861267089844)}

In [17]:
from tqdm.notebook import tqdm
svd_rmse={}
start=0
stop=200
step=1
'''
with tqdm(total=(stop-start)/step) as pbar:
    for i in range(start,stop,step):
        algo_svd = SVD(biased=False, n_factors=i)
        algo_svd = algo_svd.fit(trainset)
        svd_rmse[i]=cross_validate(algo_svd, data, verbose=False)['test_rmse'].mean()
        pbar.update(1)
dict(sorted(svd_rmse.items(), key=lambda item: item[1], reverse=False))
'''

"\nwith tqdm(total=(stop-start)/step) as pbar:\n    for i in range(start,stop,step):\n        algo_svd = SVD(biased=False, n_factors=i)\n        algo_svd = algo_svd.fit(trainset)\n        svd_rmse[i]=cross_validate(algo_svd, data, verbose=False)['test_rmse'].mean()\n        pbar.update(1)\ndict(sorted(svd_rmse.items(), key=lambda item: item[1], reverse=False))\n"

In [18]:
nmf_rmse={}
start=0
stop=200
step=1
'''
with tqdm(total=(stop-start)/step) as pbar:
    for i in range(start,stop,step):
        algo_nmf = SVD(biased=False, n_factors=i)
        algo_nmf = algo_svd.fit(trainset)
        nmf_rmse[i]=cross_validate(algo_nmf, data, verbose=False)['test_rmse'].mean()
        pbar.update(1)
dict(sorted(nmf_rmse.items(), key=lambda item: item[1], reverse=False))
'''

"\nwith tqdm(total=(stop-start)/step) as pbar:\n    for i in range(start,stop,step):\n        algo_nmf = SVD(biased=False, n_factors=i)\n        algo_nmf = algo_svd.fit(trainset)\n        nmf_rmse[i]=cross_validate(algo_nmf, data, verbose=False)['test_rmse'].mean()\n        pbar.update(1)\ndict(sorted(nmf_rmse.items(), key=lambda item: item[1], reverse=False))\n"

In [19]:
algo_svd = SVD(biased=False, n_factors=8)
algo_svd = algo_svd.fit(trainset)
cross_validate(algo_svd, data, verbose=True)

Evaluating RMSE, MAE of algorithm SVD on 5 split(s).

                  Fold 1  Fold 2  Fold 3  Fold 4  Fold 5  Mean    Std     
RMSE (testset)    0.9443  0.9409  0.9250  0.9362  0.9334  0.9360  0.0066  
MAE (testset)     0.7183  0.7209  0.7089  0.7170  0.7133  0.7157  0.0042  
Fit time          0.55    0.63    0.64    0.65    0.51    0.59    0.06    
Test time         0.11    0.14    0.21    0.13    0.11    0.14    0.04    


{'test_rmse': array([0.94426625, 0.94088049, 0.92501241, 0.93618964, 0.93340967]),
 'test_mae': array([0.71832153, 0.72088651, 0.70894828, 0.71702166, 0.7133111 ]),
 'fit_time': (0.5465395450592041,
  0.6293206214904785,
  0.6363003253936768,
  0.6492636203765869,
  0.5086686611175537),
 'test_time': (0.10970711708068848,
  0.14361357688903809,
  0.21246600151062012,
  0.13164877891540527,
  0.10672545433044434)}

In [20]:
algo_nmf = NMF(biased=False,n_factors=23)
algo_nmf = algo_nmf.fit(trainset)
cross_validate(algo_nmf, data, verbose=True)

Evaluating RMSE, MAE of algorithm NMF on 5 split(s).

                  Fold 1  Fold 2  Fold 3  Fold 4  Fold 5  Mean    Std     
RMSE (testset)    0.9269  0.9229  0.9219  0.9299  0.9282  0.9260  0.0031  
MAE (testset)     0.7041  0.6999  0.6961  0.7010  0.6987  0.7000  0.0027  
Fit time          2.61    2.62    2.94    2.62    2.61    2.68    0.13    
Test time         0.20    0.11    0.20    0.18    0.13    0.16    0.04    


{'test_rmse': array([0.9269417 , 0.92286917, 0.92190111, 0.92989443, 0.92816392]),
 'test_mae': array([0.70410916, 0.69988485, 0.69605895, 0.70102619, 0.69869902]),
 'fit_time': (2.608070135116577,
  2.620039701461792,
  2.9361257553100586,
  2.624983787536621,
  2.6096208095550537),
 'test_time': (0.20142078399658203,
  0.11066150665283203,
  0.20046401023864746,
  0.17553091049194336,
  0.13364434242248535)}

## Призрак в машине

Проинтерпретируйте получившиеся "интересы" в модели NnMF.

Например, можно начать с просмотра отсортированных по ним фильмов

In [21]:
movies = pd.read_csv('./ml-latest-small/movies.csv', delimiter=',')
# Делаем таблицу для преобразования id в имя
id_to_title = movies.loc[:, ["movieId", "title"]]
id_to_title.set_index("movieId", inplace=True)

In [22]:
factor_idx = 0
items = list(algo_nmf.trainset.all_items())

items.sort(key=lambda inner_id: -algo_nmf.qi[inner_id][factor_idx])
named_items = [[id_to_title.loc[int(trainset.to_raw_iid(k))]["title"], [int(f*100) for f in algo_nmf.qi[k]]] for k in items]
#named_items

In [23]:
for i in range (5):
    print(named_items[i][0],'\t\t',movies.loc[(movies['title'] == named_items[i][0])]['genres'].reset_index(drop=True)[0])
print("-----------")
for i in range (5,0,-1):
    print(named_items[-i][0],'\t\t',movies.loc[(movies['title'] == named_items[-i][0])]['genres'].reset_index(drop=True)[0])  

Trust (1990) 		 Comedy|Drama|Romance
Passenger, The (Professione: reporter) (1975) 		 Drama
Open Range (2003) 		 Western
Man of Steel (2013) 		 Action|Adventure|Fantasy|Sci-Fi|IMAX
Croods, The (2013) 		 Adventure|Animation|Comedy
-----------
Night in the Life of Jimmy Reardon, A (1988) 		 Comedy|Romance
Restrepo (2010) 		 Documentary|War
What Men Still Talk About (2011) 		 Comedy
Terrorist, The (a.k.a. Malli) (Theeviravaathi) (1998) 		 Drama
Pump Up the Volume (1990) 		 Comedy|Drama


## вывод
разделение по первому признаку разделяет выборку явно, с положительным знаком преимущественно комедии, детские и семейные фильмы, а с отрицательным знаком преимущественно драмы.

## Адаптация собственного алгоритма

Адаптируйте ваш алгоритм из первой лабораторной для совместимости с библиотекой согласно документации: https://surprise.readthedocs.io/en/stable/building_custom_algo.html

Сравните настроенные ранее модели со своим решением с помощью кроссвалидации.

In [32]:
from surprise import AlgoBase, Dataset
from surprise.model_selection import cross_validate
from surprise import PredictionImpossible
f=int(0)
class MyOwnAlgorithm(AlgoBase):
    
    def __init__(self):
        AlgoBase.__init__(self)
        self.f=0
        self.count=0
        self.s={}
        self.u_mid ={}
        self.ui_filmid={}
        self.ui_value={}   
    def fit(self, trainset):
        AlgoBase.fit(self, trainset)
        #if self.f==0:
        for key, value in trainset.ur.items() :
            self.ui_filmid[key]=[]
            self.ui_value[key]=[]
            for i in range (len(trainset.ur[key])):
                self.ui_filmid[key].append( value[i][0])
                self.ui_value[key].append(value[i][1])
        for key, value in trainset.ur.items() :
            s_small={}
            sum_u=0
            s_downk=0
            s_downl=0
            for k in range (len(trainset.ur[key])):
                sum_u+=trainset.ur[key][k][1]
            s_downk= sum(i*i for i in self.ui_value[key])
            for l, e in trainset.ur.items():
                if (l!=key):
                    s_up=0
                    #print(key, l)
                    s_downl=sum(i*i for i in self.ui_value[l])
                    a= list(set( self.ui_filmid[key])& set(self.ui_filmid[l]))
                    for j in a:
                        s_up+=self.ui_value[key][self.ui_filmid[key].index(j)]*self.ui_value[l][self.ui_filmid[l].index(j)]
                    s_small[l]=s_up/((s_downk*s_downl)**0.5) 
            self.s[key]=s_small
            self.u_mid[key]=sum_u/len(trainset.ur[key])
        self.f=+1
            
        return self
    def estimate(self, u, i):
        self.count+=1
        if not (self.trainset.knows_user(u) and self.trainset.knows_item(i)):
            raise PredictionImpossible("User and/or item is unknown.")
        r_up=0
        r_down=0
        for l, e in trainset.ur.items():
            if (l!=u):
                a= list(set( self.ui_filmid[u])& set(self.ui_filmid[l]))
                if a:
                    for j in a:
                        r_up+=self.s[u][l]*(self.ui_value[l][self.ui_filmid[l].index(j)]-self.u_mid[l])
                    r_down+=abs(self.s[u][l])
        if r_down==0:
            bsl=0.0
        else:
            bsl = self.u_mid[l]+r_up/r_down
        if self.count%1000==0:
            print (self.f,self.count)
        return bsl

In [33]:
#data = Dataset.load_builtin("ml-100k")
algo = MyOwnAlgorithm()


cross_validate(algo, data, verbose=True)

1 1000
1 2000
1 3000
1 4000
1 5000
1 6000
1 7000
1 8000
1 9000
1 10000
1 11000
1 12000
1 13000
1 14000
1 15000
1 16000
1 17000
1 18000
1 19000
1 20000
1 21000
1 22000
1 23000
1 24000
1 25000
1 26000
1 27000
1 28000
1 29000
1 30000
1 31000
1 32000
1 33000
1 34000
1 35000
1 36000
1 37000
1 40000
1 41000
1 42000
1 43000
1 44000
1 45000
1 46000
1 47000
1 48000
1 50000
1 52000
1 53000
1 54000
1 55000
1 56000
1 57000
1 58000
1 59000
1 60000
1 61000
1 62000
1 63000
1 64000
1 65000
1 66000
1 67000
1 68000
1 69000
1 70000
1 71000
1 72000
1 73000
1 74000
1 75000
1 76000
1 77000
1 78000
1 79000
1 80000
1 81000
1 82000
1 83000
1 84000
1 85000
1 86000
1 87000
1 88000
1 89000
1 90000
1 91000
1 92000
1 94000
1 95000
1 96000
1 97000
1 98000
1 99000
1 100000
Evaluating RMSE, MAE of algorithm MyOwnAlgorithm on 5 split(s).

                  Fold 1  Fold 2  Fold 3  Fold 4  Fold 5  Mean    Std     
RMSE (testset)    1.7322  1.7534  1.7711  1.7577  1.7471  1.7523  0.0127  
MAE (testset)     1.4051  1.4244 

{'test_rmse': array([1.73224437, 1.75340188, 1.77107993, 1.75767602, 1.74710861]),
 'test_mae': array([1.40505151, 1.42437125, 1.44093863, 1.43486618, 1.42432105]),
 'fit_time': (33.13234901428223,
  28.594185829162598,
  28.83304715156555,
  28.853599548339844,
  29.188100337982178),
 'test_time': (1212.4973239898682,
  1237.8047845363617,
  1246.427966117859,
  1248.0500223636627,
  1253.2951097488403)}

Evaluating RMSE, MAE of algorithm MyOwnAlgorithm on 5 split(s).

                  Fold 1  Fold 2  Fold 3  Fold 4  Fold 5  Mean    Std     
RMSE (testset)    1.0399  1.0551  1.0451  1.0515  1.0479  1.0479  0.0052  
MAE (testset)     0.8150  0.8297  0.8191  0.8261  0.8236  0.8227  0.0052  
Fit time          30.84   0.02    0.02    0.02    0.02    6.18    12.33   
Test time         1329.49 798.04  790.10  804.90  804.01  905.31  212.16  
{'test_rmse': array([1.03985567, 1.05505565, 1.04513776, 1.05153595, 1.04791676]),
 'test_mae': array([0.81503005, 0.82968399, 0.81909215, 0.82611232, 0.82355393]),
 'fit_time': (30.839818954467773,
  0.01994466781616211,
  0.02094554901123047,
  0.0219419002532959,
  0.02094435691833496),
 'test_time': (1329.490199804306,
  798.0431282520294,
  790.1037137508392,
  804.8991403579712,
  804.0147337913513)}

In [34]:
cross_validate(algo_svd, data, verbose=True)

Evaluating RMSE, MAE of algorithm SVD on 5 split(s).

                  Fold 1  Fold 2  Fold 3  Fold 4  Fold 5  Mean    Std     
RMSE (testset)    0.9478  0.9365  0.9408  0.9362  0.9301  0.9383  0.0059  
MAE (testset)     0.7232  0.7137  0.7206  0.7144  0.7121  0.7168  0.0043  
Fit time          0.57    0.56    0.55    0.55    0.53    0.55    0.01    
Test time         0.10    0.10    0.21    0.10    0.23    0.15    0.06    


{'test_rmse': array([0.94776565, 0.93652455, 0.94084803, 0.93620114, 0.93007406]),
 'test_mae': array([0.72316861, 0.7137201 , 0.72057928, 0.71435342, 0.71211534]),
 'fit_time': (0.566486120223999,
  0.563551664352417,
  0.551154375076294,
  0.5495352745056152,
  0.5315468311309814),
 'test_time': (0.10372233390808105,
  0.10277080535888672,
  0.20700931549072266,
  0.09974026679992676,
  0.23151159286499023)}

In [35]:
cross_validate(algo_nmf, data, verbose=True)

Evaluating RMSE, MAE of algorithm NMF on 5 split(s).

                  Fold 1  Fold 2  Fold 3  Fold 4  Fold 5  Mean    Std     
RMSE (testset)    0.9287  0.9147  0.9290  0.9332  0.9214  0.9254  0.0066  
MAE (testset)     0.6986  0.6947  0.7030  0.7068  0.6998  0.7006  0.0041  
Fit time          2.61    2.50    2.51    2.48    2.49    2.52    0.05    
Test time         0.10    0.21    0.09    0.21    0.10    0.14    0.06    


{'test_rmse': array([0.92867891, 0.91467762, 0.92904249, 0.93322884, 0.9214108 ]),
 'test_mae': array([0.69860854, 0.69470103, 0.70301195, 0.70678338, 0.69976963]),
 'fit_time': (2.606518507003784,
  2.499779224395752,
  2.5093894004821777,
  2.480039358139038,
  2.4907586574554443),
 'test_time': (0.09773111343383789,
  0.20844602584838867,
  0.09279441833496094,
  0.2124643325805664,
  0.09674954414367676)}

In [36]:
algo.predict(user_id, item_id).est

5

In [37]:
algo_nmf.predict(user_id, item_id).est

4.8589890290277085

In [38]:
algo_svd.predict(user_id, item_id).est

4.609369072951331

# Вывод
ошибка полученной модели примерно такая же, как при стандартных значениях начальной модели (n_factors default), но итоговый результат не так хорош, как хотелось бы